In [7]:
import os
import time
import cv2
from flask import Flask, render_template, Response, request, redirect, url_for, send_from_directory, flash
from object_counter import Detector, CentroidTracker, CLASS_NAMES
import imutils
import threading
APP_ROOT = os.path.dirname(os.path.abspath(__file__))
UPLOAD_FOLDER = os.path.join(APP_ROOT, "static", "uploads")
OUTPUT_FOLDER = os.path.join(APP_ROOT, "static", "outputs")
MODEL_FOLDER = os.path.join(APP_ROOT, "static", "models")
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(MODEL_FOLDER, exist_ok=True)
PROTOTXT = os.path.join(MODEL_FOLDER, "deploy.prototxt.txt")
MODEL = os.path.join(MODEL_FOLDER, "deploy.caffemodel")
app = Flask(__name__)
app.secret_key = "secret1273"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER
if not (os.path.exists(PROTOTXT) and os.path.exists(MODEL)):
    raise FileNotFoundError("Model files not found in static/models. Please add deploy.prototxt.txt and deploy.caffemodel")
detector = Detector(PROTOTXT, MODEL, conf_threshold=0.5)
output_frame = None
lock = threading.Lock()
current_counts = {"frame": {}, "unique": {}}
tracker = CentroidTracker(max_disappeared=25, max_distance=80)
def process_video_stream(vs):
    global output_frame, current_counts, tracker
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    timestamp = int(time.time())
    out_path = os.path.join(OUTPUT_FOLDER, f"processed_{timestamp}.mp4")
    writer = None
    W = None
    H = None
    while True:
        grabbed, frame = vs.read()
        if not grabbed:
            break
        if W is None or H is None:
            (H, W) = frame.shape[:2]
        boxes, class_ids, confidences = detector.detect(frame)
        ALLOWED = {"person", "car", "bicycle", "motorbike", "bus", "truck"}
        rects = []
        rect_class_names = []
        for box, cid in zip(boxes, class_ids):
            cname = CLASS_NAMES[cid] if cid < len(CLASS_NAMES) else str(cid)
            if (cname in ALLOWED) or True:
                rects.append(box)
                rect_class_names.append(cname)
        objects = tracker.update(rects)
        frame_counts = {}
        rect_centroids = []
        for (startX, startY, endX, endY) in rects:
            rect_centroids.append(((startX + endX) // 2, (startY + endY) // 2))
        object_labels = {}
        for oid, centroid in objects.items():
            min_dist = float("inf")
            label = "unknown"
            for rc_idx, rc in enumerate(rect_centroids):
                d = (centroid[0] - rc[0])**2 + (centroid[1] - rc[1])**2
                if d < min_dist:
                    min_dist = d
                    label = rect_class_names[rc_idx] if rc_idx < len(rect_class_names) else "unknown"
            object_labels[oid] = label
            frame_counts[label] = frame_counts.get(label, 0) + 1
            if oid not in tracker.counted_ids:
                tracker.counted_ids.add(oid)
                current_counts["unique"][label] = current_counts["unique"].get(label, 0) + 1
        current_counts["frame"] = frame_counts
        for idx, (startX, startY, endX, endY) in enumerate(rects):
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            label = rect_class_names[idx] if idx < len(rect_class_names) else "obj"
            text = f"{label}"
            cv2.putText(frame, text, (startX, startY - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        for oid, centroid in objects.items():
            text = f"ID {oid}:{object_labels.get(oid, '')}"
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1)
            cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 0, 255), -1)
        y = 20
        for k, v in current_counts["frame"].items():
            cv2.putText(frame, f"Now {k}: {v}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
            y += 20
        y += 5
        for k, v in current_counts["unique"].items():
            cv2.putText(frame, f"Total {k}: {v}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
            y += 20
        if writer is None:
            writer = cv2.VideoWriter(out_path, fourcc, 20, (W, H), True)
        writer.write(frame)
        with lock:
            output_frame = frame.copy()
    if writer:
        writer.release()
    vs.release()
@app.route("/", methods=["GET", "POST"])
def index():
    return render_template("index.html", uploaded=False, video=None)
@app.route("/upload", methods=["POST"])
def upload():
    if 'video' not in request.files:
        flash("No file part")
        return redirect(url_for('index'))
    file = request.files['video']
    if file.filename == '':
        flash("No selected file")
        return redirect(url_for('index'))
    filename = str(int(time.time())) + "_" + file.filename
    save_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(save_path)
    vs = cv2.VideoCapture(save_path)
    t = threading.Thread(target=process_video_stream, args=(vs,))
    t.daemon = True
    t.start()
    return render_template("index.html", uploaded=True, video=filename)
@app.route("/video_feed")
def video_feed():
    def generate():
        global output_frame
        while True:
            with lock:
                if output_frame is None:
                    continue
                (flag, encodedImage) = cv2.imencode(".jpg", output_frame)
                if not flag:
                    continue
            yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' +
                  bytearray(encodedImage) + b'\r\n')
    return Response(generate(), mimetype='multipart/x-mixed-replace; boundary=frame')
@app.route("/download/<path:filename>")
def download_file(filename):
    return send_from_directory(app.config['OUTPUT_FOLDER'], filename, as_attachment=True)
if __name__ == "__main__":
    app.run(debug=True)



# html code
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Smart Object Counter</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
    <style>
        body { background: #111; color: #eee; font-family: Arial, sans-serif; text-align: center; padding: 30px; }
        .card { background:#1a1a1a; padding:20px; border-radius:12px; box-shadow: 0 0 20px #000; width: 80%; margin:auto; }
        input[type=file] { margin-top: 10px; }
        button { margin-top: 10px; padding:10px 18px; border-radius:8px; border:none; background:#00eaff; color:#000; font-weight:bold; cursor:pointer;}
        .video-stream { margin-top: 15px; border-radius: 8px; box-shadow: 0 0 12px #00eaff; }
    </style>
</head>
<body>
    <div class="card">
        <h1>Smart Object Counter</h1>
        <p>Upload a video to count objects (or use webcam - see advanced options).</p>

        <form action="{{ url_for('upload') }}" method="post" enctype="multipart/form-data">
            <input type="file" name="video" accept="video/*" required>
            <br>
            <button type="submit">Upload & Start Processing</button>
        </form>

        {% if uploaded %}
            <h3>Processing started for: {{ video }}</h3>
            <p>Live processed video below:</p>
            <img class="video-stream" src="{{ url_for('video_feed') }}" width="720" />
            <p>Processed video will also be saved in <code>static/outputs</code>.</p>
        {% endif %}

        <hr style="border-color:#222; margin-top:25px;">
        <p style="font-size:0.9em; color:#bbb;">Tip: For better counting, use a stationary camera and clear view of the scene.</p>
    </div>
</body>
</html>


ModuleNotFoundError: No module named 'object_counter'